In [527]:
import os
import json

import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import date
import requests
from ast import literal_eval

from bqplot import Lines, Figure, LinearScale, DateScale, Axis, Hist

from ipyleaflet import Map, GeoJSON, WidgetControl, ImageOverlay
from ipywidgets import Layout, Dropdown

In [528]:
data = pd.read_json(os.path.abspath("nations.json"))

# res = requests.get("https://raw.githubusercontent.com/RichardQZeng/Notebooks/main/footprint_fixed.geojson")
# gdf = gpd.read_file(res.text)
gdf = gpd.read_file(r"C:\BERATools\Notebooks\footprint_fixed-4326.geojson")

In [529]:
def array_eval(x):
    array_str = ','.join(x.split())
    array_str = array_str.replace('[,', '[')
    array_str = array_str.replace('\n', '')
    return np.array(literal_eval(array_str))
    
gdf['sampling_widths'] = gdf['sampling_widths'].apply(array_eval)

In [530]:
gdf['OLnFID']

0        6
1        0
2        4
3        2
4        1
      ... 
202    131
203    206
204    180
205    155
206    200
Name: OLnFID, Length: 207, dtype: int64

In [531]:
def clean_data(data):
    for column in ["income", "lifeExpectancy", "population"]:
        data = data.drop(data[data[column].apply(len) <= 4].index)
    return data


def extrap_interp(data):
    data = np.array(data)
    x_range = np.arange(1800, 2009, 1.0)
    y_range = np.interp(x_range, data[:, 0], data[:, 1])
    return y_range


def extrap_data(data):
    for column in ["income", "lifeExpectancy", "population"]:
        data[column] = data[column].apply(extrap_interp)
    return data

In [532]:
data = clean_data(data)
data = extrap_data(data)

In [533]:
data

,name,region,income,population,lifeExpectancy
0,Angola,Sub-Saharan Africa,"[359.93, 359.93, 359.93, 359.93, 359.93, 359.9...","[1567028.0, 1567028.0, 1567028.0, 1567028.0, 1...","[26.98, 26.98, 26.98, 26.98, 26.98, 26.98, 26...."
1,Benin,Sub-Saharan Africa,"[553.72, 553.72, 553.72, 553.72, 553.72, 553.7...","[636559.0, 636559.0, 636559.0, 636559.0, 63655...","[31.0, 31.0, 31.0, 31.0, 31.0, 31.0, 31.0, 31...."
2,Botswana,Sub-Saharan Africa,"[407.36, 407.36, 407.36, 407.36, 407.36, 407.3...","[121000.0, 121000.0, 121000.0, 121000.0, 12100...","[33.6, 33.6, 33.6, 33.6, 33.6, 33.6, 33.6, 33...."
3,Burkina Faso,Sub-Saharan Africa,"[454.33, 454.33, 454.33, 454.33, 454.33, 454.3...","[1665421.0, 1665421.0, 1665421.0, 1665421.0, 1...","[29.2, 29.2, 29.2, 29.2, 29.2, 29.2, 29.2, 29...."
4,Burundi,Sub-Saharan Africa,"[447.59, 447.59, 447.59, 447.59, 447.59, 447.5...","[899097.0, 899097.0, 899097.0, 899097.0, 89909...","[31.5, 31.5, 31.5, 31.5, 31.5, 31.5, 31.5, 31...."
...,...,...,...,...,...
174,Thailand,East Asia & Pacific,"[496.98, 496.98, 496.98, 496.98, 496.98, 496.9...","[4665000.0, 4665000.0, 4665000.0, 4665000.0, 4...","[30.4, 30.4, 30.4, 30.4, 30.4, 30.4, 30.4, 30...."
175,Timor-Leste,East Asia & Pacific,"[514.12, 514.3505, 514.581, 514.8115, 515.042,...","[137262.0, 137262.0, 137262.0, 137262.0, 13726...","[28.97, 28.97, 28.97, 28.97, 28.97, 28.97, 28...."
177,Tonga,East Asia & Pacific,"[667.71, 667.71, 667.71, 667.71, 667.71, 667.7...","[18658.0, 18654.325581395347, 18650.6511627907...","[57.91, 57.91, 57.91, 57.91, 57.91, 57.91, 57...."
178,Vietnam,East Asia & Pacific,"[459.71, 459.71, 459.71, 459.71, 459.71, 459.7...","[6551000.0, 6551000.0, 6551000.0, 6551000.0, 6...","[32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32.0, 32...."


In [534]:
date_start =date(1800, 12, 31)
date_end = date(2009, 12, 31)

date_scale = DateScale(min=date_start, max=date_end)

In [535]:
date_data = pd.date_range(start=date_start, end=date_end, freq="A", normalize=True)

C:\Users\qingyerichard.zeng\AppData\Local\Temp\ipykernel_22712\2910955754.py:1: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  date_data = pd.date_range(start=date_start, end=date_end, freq="A", normalize=True)


In [536]:
country_name = "Angola"
data_name = "income"

line_id = 0

# x_data = data[data.name == country_name][data_name].values[0]
x_data = gdf[gdf['OLnFID'] == 0]['sampling_widths'].values[0]

In [537]:
x_data

array([3.01943204, 3.01943204, 5.03238673, 5.03238673, 5.03238673,
       5.03238673, 6.03886408, 6.03886408, 5.03238673, 5.03238673,
       5.03238673, 6.03886408, 5.03238673, 5.03238673, 5.03238673,
       6.03886408, 6.03886408, 6.03886408, 6.03886408, 5.03238673,
       5.03238673, 5.03238673, 6.03886408, 6.03886408, 6.03886408,
       6.03886408, 6.03886408, 7.04534142, 6.03886408, 6.03886408,
       6.03886408, 6.03886408, 6.03886408, 6.03886408, 5.03238673,
       4.02590938])

In [538]:
x_sc = LinearScale()
y_sc = LinearScale()

hist = Hist(sample=x_data, scales={"sample": x_sc, "count": y_sc})
ax_x = Axis(scale=x_sc, tick_format="0.2f")
ax_y = Axis(scale=y_sc, orientation="vertical")
hist.bins = 10

figure = Figure(marks=[hist], axes=[ax_x, ax_y], padding_y=0)

In [539]:
def update_figure(line_id):
    hist.sample = gdf[gdf['OLnFID'] == line_id]['sampling_widths'].values[0]
    ax_y.label = str(line_id)

In [540]:
country_name = "Angola"
data_name = "population"

update_figure(line_id)

In [541]:
with open("./countries.geo.json") as f:
    countries = json.load(f)

In [542]:
with open(r"C:\BERATools\Notebooks\footprint_fixed-4326.geojson") as f:
    lines = json.load(f)

with open(r"C:\BERATools\Notebooks\footprint-4326.geojson") as f:
    lines_org = json.load(f)

In [543]:
image = ImageOverlay(url=r"./CHM-4326.tif", bounds=((56.187, -110.693), (56.214, -110.645)))

In [544]:
m = Map(center=(56.20112069, -110.67053897),
        zoom=14,
        scroll_wheel_zoom=True,
        layout=Layout(height='700px'))

In [545]:
geo = GeoJSON(
    data=lines,
    style={"fillColor": "white", "weight": 0.5, 'opacity': 0.5,},
    name="Line footprint",
)
m.add(geo)

geo_org = GeoJSON(
    data=lines_org,
    style={"fillColor": "rgb(54, 49, 92)", "weight": 1},
    hover_style={"fillColor": "#148ee6"},
    name="Line footprint",
)
m.add(geo_org)

m.add_layer(image);

widget_control1 = WidgetControl(widget=figure, position="bottomright")
widget_control1.min_width = 400

m.add(widget_control1)


def on_hover(event, feature, **kwargs):
    global line_id

    line_id = feature["properties"]["OLnFID"]
    update_figure(line_id)
    print("feature", feature)


geo.on_hover(on_hover)

feature {'type': 'Feature', 'properties': {'OLnFID': 118, 'feature_ty': None, 'year': 0, 'source': None, 'shape_leng': 0, 'BT_UID': 44, 'DynCanTh': 2.271999990567565, 'OLnSEG': 0, 'L_Pertile': 13.816999912261963, 'R_Pertile': 13.927000045776367, 'CL_CutHt': 0.522499989718199, 'CR_CutHt': 4.021499991416931, 'RDist_Cut': 6, 'LDist_Cut': 6, 'avg_width': 13.002703688405402, 'max_width': 13.40658777108026, 'sampling_widths': '[12.53959448 12.00249571 15.7885981  15.49472126 15.17289343 14.66006209\n 10.50010399 13.83330113 13.44261428 11.00228774 12.00249571 13.00270369\n 13.00270369 13.00270369 13.00270369 10.00207976 10.00207976 11.00228774\n 11.00228774 11.00228774 10.00207976  9.00187178  9.00187178 10.00207976\n 10.00207976 10.00207976 10.00207976 10.00207976  9.00187178  9.00187178\n  8.00166381  8.00166381  8.00166381  3.00062393  5.00103988  5.00103988\n  5.00103988  7.00145583  6.00124786  6.00124786  8.00166381  7.00145583\n  9.00187178 10.00207976 12.00249571 13.26248173 13.00270

In [546]:
dropdown = Dropdown(
    options=["income", "population", "lifeExpectancy"],
    value=data_name,
    description="Plotting:",
)


def on_click(change):
    global footrpint_type

    footrpint_type = change["new"]
    print(change["new"])
    # update_figure(footrpint_type, line_id)


dropdown.observe(on_click, "value")

widget_control2 = WidgetControl(widget=dropdown, position="bottomleft")

# m.add(widget_control2)

In [547]:
m

Map(center=[56.20112069, -110.67053897], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_t…